In [1]:
!head raw_fbank_pitch_all_181506.1.scp

--7E2sU6zP4_10 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:15
--7E2sU6zP4_11 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:49458
--7E2sU6zP4_12 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:125776
--7E2sU6zP4_13 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:172983
--7E2sU6zP4_5 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:192669
--7E2sU6zP4_6 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:221557
--7E2sU6zP4_7 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:275815
--7E2sU6zP4_8 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:314937
--7E2sU6zP4_9 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:364164
--8pSDeC-fg_0 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:396664


In [1]:
vid_ids = []

with open('cmvn_all_181506.scp') as f:
    for line in f:
        vid, path = line.split()
        vid_ids.append(vid)

In [2]:
len(vid_ids)

13668

In [3]:
vid_ids = set(vid_ids)

In [7]:
import sys
import os

"""Replaces paths in Kaldi .scp files with a given placeholder.
Reads from STDIN and writes to STDOUT."""

try:
    pattern = '/home/aman_khullar/how2/fbank_pitch_181506'
except IndexError as ie:
    print('Usage: {} <placeholder string>'.format(sys.argv[0]))
    sys.exit(1)

for idx in range(1, 11):
    print("Processing file at idx", idx)
    with open('raw_fbank_pitch_all_181506.{}.scp'.format(idx)) as f, open('raw_fbank_pitch_all_181506.{}_processed.scp'.format(idx), 'w') as fw:
        for line in f:
            line = line.strip()
            utt, path = line.split()
            ark_file, offset = path.split(':')
            ark_file = ark_file.split('/')[-1]
            fw.write('{} {}/{}:{}\n'.format(utt, pattern, ark_file, offset))

Processing file at idx 1
Processing file at idx 2
Processing file at idx 3
Processing file at idx 4
Processing file at idx 5
Processing file at idx 6
Processing file at idx 7
Processing file at idx 8
Processing file at idx 9
Processing file at idx 10


In [5]:
audio_feats = {}

from kaldiio import ReadHelper
for fidx in range(1, 11):
    print("Evaluating file", fidx)
    with ReadHelper('scp:raw_fbank_pitch_all_181506.{}_processed.scp'.format(fidx)) as reader:
        for idx, (key, numpy_array) in enumerate(reader):
            audio_feats[key] = numpy_array

Evaluating file 1
Evaluating file 2
Evaluating file 3
Evaluating file 4
Evaluating file 5
Evaluating file 6
Evaluating file 7
Evaluating file 8
Evaluating file 9
Evaluating file 10


In [6]:
print(len(audio_feats))

191608


In [7]:
from collections import defaultdict

vid_utts = defaultdict(list)

for key in audio_feats.keys():
    try:
        curr_utt_ids = []
        # To manage utts of format vid_1_2_3
        while True:
            split_idx = key.rfind('_')
            vid_id, utt_id = key[:split_idx], key[split_idx+1:]
            curr_utt_ids.append(utt_id)
            if vid_id in vid_ids:
                for utt_id in curr_utt_ids:
                    vid_utts[vid_id].append(int(utt_id))
                break
            key = vid_id
    except:
        print("Error at key:",key)
        break

In [8]:
len(vid_utts)

13668

In [9]:
vid_utts['-00cp1iGiDw']

[1, 11, 10, 12, 13, 14, 15, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
for key in vid_utts:
    vid_utts[key] = sorted(vid_utts[key])

In [55]:
# 00cp1iGiDw_10_11

# for key in vid_utts.keys():
#     if '_' in key:
#         print(key)
#         print(vid_utts[key])
#         break

--dANj_01AU
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [11]:
combined_feats = defaultdict(list)

for vid_id in vid_utts:
    utt_idx = 0
    key = vid_id
    # To manage utts of format vid_1_2_3
    while utt_idx < len(vid_utts[vid_id]):
        utt_id = vid_utts[vid_id][utt_idx]
        key = '{}_{}'.format(key, utt_id)
        if key in audio_feats:
            combined_feats[vid_id].append(audio_feats[key])
            key = vid_id
        utt_idx += 1

In [12]:
len(combined_feats)

13668

In [16]:
import numpy as np

concat_feats = {}

for key in combined_feats:
    concat = np.vstack(combined_feats[key])
    concat_feats[key] = concat

In [17]:
print(len(concat_feats))

13668


In [19]:
for key, val in concat_feats.items():
    np.save('concat/{}.npy'.format(key), val)

In [4]:
import os

test_vid_ids = set()

for fname in os.listdir('/home/aman_khullar/how2/video_features_300/'):
    test_vid_ids.add(fname[:-4])

In [8]:
len(test_vid_ids.intersection(vid_ids))

13445